In [ ]:
import pandas as pd
from edisgo.edisgo import import_edisgo_from_files
from edisgo.flex_opt.costs import grid_expansion_costs
from edisgo.tools.tools import get_sample_using_time
from copy import deepcopy
import os

In [ ]:
grid = "5bus_testcase"
path = os.path.join(".", "edisgo_scenario_data", grid)
solution_path = os.path.join(".","opf_solutions", grid+".csv")

In [ ]:
edisgo = import_edisgo_from_files(path, import_topology=True,
                                  import_timeseries=True,
                                  import_electromobility=True,
                                  import_heat_pump=True,
                                  import_dsm=True)
psa_net = edisgo.to_pypsa()
edisgo_OPF = deepcopy(edisgo)
edisgo.reinforce()
cost = grid_expansion_costs(edisgo)

# 1. Flexibilitäten
Implementiert sind folgende Flexibilitäten
* Elektromobilität
* Wärmepumpen mit Wärmespeichern
* Batteriespeicher
* Demand Side Management

Damit die flexiblen Verbraucher in der Julia-Optimierung berücksichtigt werden, müssen für jede Flexibilität eine Liste der Namen der flexiblen Verbraucher an den entsprechenden Parameter in ```edisgo.opf.powermodels_opf.pm_optimize()``` übergeben. 

Parameter:
* flexible_cps
* flexible_hps
* flexible_storages
* flexible_loads

Falls Flexibilität nicht mit optimiert werden soll, den Parameter nicht setzen. 

In [ ]:
# Elektromobilität
flexible_cps = psa_net.loads.loc[
    (psa_net.loads.index.str.contains("home"))| 
    (psa_net.loads.index.str.contains("work"))
].index.values
# Wärmepumpen/-speicher
flexible_hps = edisgo.heat_pump.thermal_storage_units_df.index.values
# Batteriespeicher
flexible_storages = edisgo.topology.storage_units_df.index.values
# DSM
flexible_loads = edisgo.dsm.p_max.columns

# 2. OPF Version
Parameter: opf_version
1. Mit Anforderungen aus dem übergelagerten Netz, ohne Netzrestriktionen
2. Mit Anforderungen aus dem übergelagerten Netz, mit Netzrestriktionen
3. Ohne Anforderungen aus dem übergelagerten Netz, ohne Netzrestriktionen
4. Ohne Anforderungen aus dem übergelagerten Netz, mit Netzrestriktionen

In [ ]:
opf_version = 3
# Für OPF mit Anforderungen aus dem übergelagerten Netz müssen momentan noch Dummy Daten erzeugt werden
if opf_version in [1,2]:
    cps = psa_net.loads.loc[psa_net.loads.index.str.contains("Charging")].index.values
    hps = psa_net.loads.loc[psa_net.loads.index.str.contains("HP_Load")].index.values
    edisgo_OPF.overlying_grid.dsm_active_power = psa_net.loads_t.p_set.loc[:, edisgo_OPF.dsm.p_max.columns].sum(axis=1)
    edisgo_OPF.overlying_grid.electromobility_active_power = psa_net.loads_t.p_set.filter(cps).sum(axis=1)
    edisgo_OPF.overlying_grid.heat_pump_rural_active_power = psa_net.loads_t.p_set.filter(hps).sum(axis=1)
    edisgo_OPF.overlying_grid.heat_central_active_power = pd.Series(index=edisgo_OPF.timeseries.timeindex,
                                                      data=[0] * len(edisgo_OPF.timeseries.timeindex))
    edisgo_OPF.overlying_grid.renewables_curtailment = pd.Series(index=edisgo_OPF.timeseries.timeindex,
                                                      data=[0.0001] * len(edisgo_OPF.timeseries.timeindex))
    edisgo_OPF.overlying_grid.storage_units_active_power = psa_net.storage_units_t.p_set.sum(axis=1)

# 3. Optimierungsmethode
Parameter: method
- "soc": konvexe SOC-Optimierung
- "nc": nicht-konvexe Optimierung

In [ ]:
method = "soc"

# 4. eDisGo Funktionen für die Optimierung
## 4.1. ```to_powermodels()```
Wandelt eDisGo Objekt in PowerModels Network Data Format um.  
```python
to_powermodels(
    self,
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
    )
```

In [ ]:
pm, hv_flex_dict = edisgo_OPF.to_powermodels(flexible_cps=flexible_cps, flexible_hps=flexible_hps,
                               flexible_loads=flexible_loads, flexible_storages=flexible_storages,
                               opf_version=opf_version)
pm

## 4.3. ```save_edisgo_to_json()```
Speichert eDisGo Objekt im PowerModels Network Data Format als json. Wird zum Debuggen mit Julia benötigt.
```python
save_edisgo_to_json(
    self,
    filename=None,
    path="",
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
)

```

## 4.4. ```pm_optimize()```
Startet die Julia Optimierung für das eDisGo Objekt in einem Subproccess und schreibt die Ergebnisse der Optimierung wieder auf das eDisGo Objekt.
```python
pm_optimize(
    self,
    s_base=1,
    flexible_cps=None,
    flexible_hps=None,
    flexible_loads=None,
    flexible_storages=None,
    opf_version=4,
    method="soc",
    warm_start=False,
    silence_moi=False,
    save_heat_storage=False,
    save_slack_gen=False,
    save_slacks=False,
    path="",
)

```

In [ ]:
edisgo_OPF.pm_optimize(flexible_cps=flexible_cps,
                       flexible_hps=flexible_hps,
                       flexible_loads=flexible_loads,
                       flexible_storages=flexible_storages,
                       s_base=1,
                       opf_version=opf_version, 
                       silence_moi=False);

## 4.2. ```from_powermodels()```
Schreibt Ergebnisse der PowerModels Optimierung (Zeitreihen für die Flexibilitäten) zurück auf das eDisGo Objekt. Wird in Funktion ```pm_optimize()``` beutzt. 
```python
from_powermodels(
    self,
    pm_results,
    hv_flex_dict,
    s_base=1,
    save_heat_storage=False,
    save_slack_gen=False,
    save_slacks=False,
    path="",
)
```

In [ ]:
edisgo_OPF.reinforce()
cost_OPF = grid_expansion_costs(edisgo_OPF)

# 5. Beispiel: Netz 1056

In [ ]:
grid = "1056"
path = os.path.join(".", "edisgo_scenario_data", grid)
solution_path = os.path.join(".","opf_solutions", grid+".csv")
grid_1056 = import_edisgo_from_files(path, import_topology=True,
                                  import_timeseries=True,
                                  import_electromobility=True,
                                  import_heat_pump=True,
                                  import_dsm=True)

Optimierung nur für einen Teilzeitraum laufen lassen:
```python
edisgo.tools.tools.get_sample_using_time(edisgo, start_date, end_date)
```

In [ ]:
psa_net = grid_1056.to_pypsa()
grid_1056_OPF = deepcopy(grid_1056)
grid_1056.reinforce()
cost_1056 = grid_expansion_costs(grid_1056)

In [ ]:
# Elektromobilität
flexible_cps = psa_net.loads.loc[
    (psa_net.loads.index.str.contains("home"))| 
    (psa_net.loads.index.str.contains("work"))
].index.values
# Wärmepumpen/-speicher
flexible_hps = grid_1056_OPF.heat_pump.thermal_storage_units_df.index.values
# Batteriespeicher
flexible_storages = grid_1056_OPF.topology.storage_units_df.index.values
# DSM
flexible_loads = grid_1056_OPF.dsm.p_max.columns

In [ ]:
grid_1056_OPF.pm_optimize(flexible_cps=flexible_cps,
                       flexible_hps=flexible_hps,
                       flexible_loads=flexible_loads,
                       flexible_storages=flexible_storages,
                       s_base=1,
                       opf_version=opf_version, 
                       silence_moi=False);

In [ ]:
grid_1056_OPF.reinforce()
cost = grid_expansion_costs(grid_1056)